# Not Scaled Data

In [108]:
import pandas as pd
xls = pd.ExcelFile("Фрустрация.xlsx")
data = pd.read_excel(xls, xls.sheet_names)    

In [109]:
keys = [' E', ' E1', ' e_', ' M', ' M1', ' m_', ' I', ' I1', ' i_']

keys1 = [[' E', ' E1', ' e_'],
         [' M', ' M1', ' m_'],
         [' I', ' I1', ' i_']]

keys2 = [[' E', ' M', ' I'],
        [' E1', ' M1', ' I1'],
        [' e_', ' m_', ' i_']]
def get_y(keys, data, init_keys = keys):
    y = []
    for ind, i in enumerate(init_keys):
        for ind1, j in enumerate(keys):
            if i in j:
                y.append(ind1 * np.ones(len(data[i])))
    return np.hstack(y)

In [110]:
data[keys[0]].columns

Index(['Unnamed: 0', 'Число восклицательных знаков',
       'Число вопросительных знаков', 'Число слов', 'Число символов',
       'Число уникальных слов', 'Число клауз', 'Число предложений',
       'Число знаков препинания',
       'Средняя длина предложения (в количестве слов)',
       ...
       'Синтакс. связь: nummod', 'Синтакс. связь: nummod:gov',
       'Синтакс. связь: obj', 'Синтакс. связь: obl', 'Синтакс. связь: orphan',
       'Синтакс. связь: parataxis', 'Синтакс. связь: punct',
       'Синтакс. связь: root', 'Синтакс. связь: xcomp',
       'Синтакс. связь: compound'],
      dtype='object', length=243)

In [113]:
def extract(list_of_texts):
    res = []
    for s in list_of_texts:
        k = 0
        for c in s:
            if c.islower():
                k += 1
        res.append(k/len(s))
    
    return pd.DataFrame({"Доля":res})

In [114]:
import json
with open("p3_texts.json", "r") as f:
    texts = f.read()
    texts = json.loads(texts)
    f.close()
print(texts.keys())
for key in keys:
    if key[-1] == '1':
        key1 = key[:-1] + "'"
    else:
        key1 = key
    data[key] = data[key].join( extract(texts['P3:' + key1]), rsuffix = "___")

dict_keys(['P3: E', 'P3: M', 'P3: m_', 'P3: e_', "P3: I'", "P3: E'", 'P3: i_', "P3: M'", 'P3: инф', 'P3: I', 'P3: n ("надо")'])


In [115]:
import numpy as np
from sklearn.preprocessing import scale
X = np.vstack([data[i].values for i in keys])
X = X[:, 1:]
y1 = get_y(keys1, data)
y2 = get_y(keys2, data)

In [116]:
print("Число слов", X[:, 2].mean())

Число слов 36.49063670411985


In [117]:
print("Число предложений", X[:, 6].mean())

Число предложений 3.4569288389513106


In [118]:
print("Число символов", X[:, 3].mean())

Число символов 188.49812734082397


In [119]:
X = scale(X)
X_bin = np.where(X <= 0, 0, 1)

## The First Union

In [121]:
from sklearn.metrics import f1_score
p1 = 0.1
p2 = 0.2
M = None
N = 100
for p1 in np.linspace(0, 1, 10):
    for p2 in np.linspace(0, 1 - p1, 10):
        s = 0
        for k in range(N):
            y_pred = np.random.choice([0, 1, 2], len(y2), p=[p1, p2, 1 - p1 - p2])
            y1_ = shuffle(y1)
            s += f1_score(y1_, y_pred, average = 'macro')
        s = s/N
        if M is None or s > M[1]:
            M = ([p1, p2, 1- p1 - p2], s)
M

([0.7777777777777777, 0.19753086419753096, 0.024691358024691357],
 0.33525947944453804)

### 1. Not binarized data

In [73]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

s = 0
N = 100
for i in range(N):
    X_, y = shuffle(X, y1)
    model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
    s+= cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()
s/N

/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.38424818817232087

In [68]:
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

X_ = PCA(1 - 1e-3).fit(X_).transform(X_)
print(X_.shape)
s = 0
N = 100
for i in range(N):
    X_, y = shuffle(X, y1)
    model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
    s += cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()
s/N

(267, 157)


/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/elias/anaconda3/lib/python

0.38672091708434764

### 2. Binarized data

In [74]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

X_, y = shuffle(X_bin, y1)
print(X_.shape)
s = 0
N = 100
for i in range(N):
    X_, y = shuffle(X_bin, y1)
    model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
    s+=cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()
s/N

(267, 243)


/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/elias/anaconda3/lib/python

0.3609918098816389

In [75]:
from sklearn.linear_model import LogisticRegression

X_ = PCA(1 - 1e-3).fit(X_).transform(X_)
print(X_.shape)
s = 0
N = 100
for i in range(N):
    X_, y = shuffle(X_bin, y1)
    model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
    s += cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()
s/N

(267, 157)


/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/elias/anaconda3/lib/python

0.3592004363020605

## The Second Union

In [122]:
from sklearn.metrics import f1_score
p1 = 0.1
p2 = 0.2
M = None
N = 100
for p1 in np.linspace(0, 1, 10):
    for p2 in np.linspace(0, 1 - p1, 10):
        s = 0
        for k in range(N):
            y_pred = np.random.choice([0, 1, 2], len(y2), p=[p1, p2, 1 - p1 - p2])
            y2_= shuffle(y2)
            s += f1_score(y2_, y_pred, average = 'macro')
        s = s/N
        if M is None or s > M[1]:
            M = ([p1, p2, 1- p1 - p2], s)
M

([0.6666666666666666, 0.14814814814814817, 0.1851851851851852],
 0.33284173012414386)

### 1. Not binarized data

In [78]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

X_, y = shuffle(X, y2)
print(X_.shape)
s = 0
N = 100
for i in range(N):
    X_, y = shuffle(X, y2)
    model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
    s += cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()
s/N

(267, 243)


0.35973429903519116

In [79]:
from sklearn.linear_model import LogisticRegression

X_ = PCA(1 - 1e-3).fit(X_).transform(X_)
print(X_.shape)
s = 0
N = 100
for i in range(N):
    X_, y = shuffle(X, y2)
    model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
    s += cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()
s/N

(267, 157)


0.3597259923214435

### 2. Binarized data

In [81]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

s = 0
N = 100
for i in range(N):
    X_, y = shuffle(X_bin, y2)
    model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
    s += cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()
s/N

0.35302800634456477

In [89]:
from sklearn.linear_model import LogisticRegression

X_ = PCA(1 - 1e-3).fit(X_).transform(X_)
print(X_.shape)
s = 0
N = 100
for i in range(N):
    X_, y = shuffle(X_bin, y2)
    model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
    s += cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()
s/N

(267, 157)


0.3517928010536153